In [1]:
import cv2 as cv
import os

In [2]:
camera = cv.VideoCapture(0)

In [4]:
labels = ["hard_hat_on","hard_hat_off","hard_hat_on_test","hard_hat_off_test"]
for label in labels:
    if not os.path.exists(label):
        os.mkdir(label)

In [5]:
for folder in labels[:2]:
    print("Press any button to start data collection for"+folder)
    userinput = input()  
    for count in range(200):
        status, frame = camera.read()
        yep = frame
        cv.imshow("Video Window",yep)
        frame = cv.resize(yep, (224,224))
        cv.imwrite('C:/Users/Cameron/capstone/'+folder+'/img'+str(count)+'.png',yep)
        if cv.waitKey(1) == ord('q'):
            break
for folder in labels[2:]:
    print("Press any button to start data collection for"+folder)
    userinput = input()  
    for count in range(20):
        status, frame = camera.read()
        yep = frame
        cv.imshow("Video Window",yep)
        frame = cv.resize(yep, (224,224))
        cv.imwrite('C:/Users/Cameron/capstone/'+folder+'/img'+str(count)+'.png',yep)
        if cv.waitKey(1) == ord('q'):
            break            
# When everything done, release the capture
camera.release()
cv.destroyAllWindows()

Press any button to start data collection forhard_hat_on
 
Press any button to start data collection forhard_hat_off
 
Press any button to start data collection forhard_hat_on_test
s
Press any button to start data collection forhard_hat_off_test
 


In [42]:
from PIL import Image
from skimage import io
import numpy as np
image = Image.open('C:/Users/Cameron/capstone/hard_hat/img1.png')
io.imshow(np.asarray(image))

In [6]:
#Dataset
from torch.utils.data import Dataset, DataLoader
import os
import imageio
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self,img_dir_true,img_dir_false):
        self.img_dirs = [os.path.join(img_dir_true,img) for img in os.listdir(img_dir_true)] + [os.path.join(img_dir_false,img) for img in os.listdir(img_dir_false)]
        self.labels = [1]*len(os.listdir(img_dir_true)) + [0]*len(os.listdir(img_dir_false))
        print(len(self.labels))
        print(len(self.img_dirs))
    def __len__(self):
        return len(self.img_dirs)

    def __getitem__(self, idx):   
        batch = {"data" : torch.transpose(torch.tensor(imageio.imread(self.img_dirs[idx])),0,2).float() , "labels" : torch.tensor(self.labels[idx])}        
        return [batch]

In [7]:
dataset = CustomImageDataset(img_dir_true = "C:/Users/Cameron/capstone/hard_hat_on", img_dir_false = "C:/Users/Cameron/capstone/hard_hat_off")
test_dataset = CustomImageDataset(img_dir_true = "C:/Users/Cameron/capstone/hard_hat_on_test", img_dir_false = "C:/Users/Cameron/capstone/hard_hat_off_test")

400
400
40
40


In [8]:
import torch
train_set, val_set = torch.utils.data.random_split(dataset, [320, 80])

In [9]:
import torchvision
model =  torchvision.models.mobilenet_v2(pretrained=True)

In [10]:
model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features,out_features=1)
#model.classifier[2] = 

In [11]:
#lightning module
import pytorch_lightning as pl
import torchvision
F = torch.nn.BCEWithLogitsLoss()
M = torch.nn.Sigmoid()

class HatNoHat(pl.LightningModule):
    def __init__(self, train_dataset, val_dataset, model, col_fn, learning_rate=5e-5, num_loading_cpus=1, batch_size=4):
        super().__init__() 
        self.model =  model
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.col_fn = col_fn
        self.learning_rate = learning_rate
        self.num_loading_cpus = num_loading_cpus
        self.batch_size = batch_size
    
    def forward(self,x):
        f = self.model.forward(x)
        return f
    
    def training_step(self, batch, batch_idx):
        x = batch['data']
        #print('x ', x)
        y = torch.unsqueeze(batch['labels'],0).float()
        #print('y ', y)
        y_hat = self.forward(x.float())
        print('y_hat ', y_hat)
        output = F(y_hat,y)
        #print('loss ', output)
        return output
        
    def validation_step(self, batch, batch_idx):
        x = batch['data']
        #print('x ', x)
        y = torch.unsqueeze(batch['labels'],0).float()
        #print('y ', y)
        y_hat = self.forward(x.float())
        #print('y_hat ', y_hat)
        output = F(y_hat,y)
        #print('loss ', loss)
        return output
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset,batch_size=self.batch_size,collate_fn=self.col_fn)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset,batch_size=self.batch_size,collate_fn=self.col_fn)     
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
    

In [12]:
import torch

def col_fn(batch):
    #print(batch)
    out = dict()
    out['data'] = torch.stack([x['data'] for x in batch[0]])
    out['labels'] = torch.stack([x['labels'] for x in batch[0]])
    #print(out)
    return out

In [13]:
agh = HatNoHat(train_set, val_set, model, col_fn) 

In [14]:
trainer = pl.Trainer(max_epochs=2)
trainer.fit(agh)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type        | Params
--------------------------------------
0 | model | MobileNetV2 | 2.2 M 
--------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.901     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\Cameron\.conda\envs\camdefaultenv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Cameron\.conda\envs\camdefaultenv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

y_hat  tensor([[-0.0706]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1483]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1338]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1103]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.2087]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.4283]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.0372]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.1371]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.0314]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.3076]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1407]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.4841]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.0026]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1687]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1899]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.2902]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.2471]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1628]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1900]], grad_fn=<AddmmBackward0>)
y_hat  t

Validating: 0it [00:00, ?it/s]

y_hat  tensor([[0.2544]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.4545]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.3039]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.2708]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.6687]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.5266]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.2282]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.3060]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.0047]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.2821]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.1937]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.4746]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.1790]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.2932]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.4412]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.2737]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.5497]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[0.2641]], grad_fn=<AddmmBackward0>)
y_hat  tensor([[-0.4274]], grad_fn=<AddmmBackward0>)
y_hat 

Validating: 0it [00:00, ?it/s]

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
dl = torch.utils.data.DataLoader(val_set,batch_size=1,collate_fn=col_fn)
M = torch.nn.Sigmoid()
pred = []
labels = []
with torch.no_grad():
    for i,batch in enumerate(dl):
        plt.figure()
        #if i == 5:
        plt.imshow(np.asarray(torch.transpose(batch['data'].squeeze().long(),0,2)))
        plt.show()
        x = batch['data']
        y = torch.unsqueeze(batch['labels'],0).float()
        #print('y ', y)
        y_hat = model.forward(x.float())
        print('y_hat ', M(y_hat)   )  
        if y_hat.numpy().tolist()[0][0] > 0.5:
            print('HAT')
            pred.append(1)
        else:
            print("NO HAT")
            pred.append(0)
        labels.append(y.numpy().tolist()[0][0])

In [16]:
from sklearn.metrics import precision_recall_fscore_support

In [17]:
precision_recall_fscore_support(labels,pred)

(array([0.95744681, 1.        ]),
 array([1.        , 0.94285714]),
 array([0.97826087, 0.97058824]),
 array([45, 35], dtype=int64))

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
dl = torch.utils.data.DataLoader(test_dataset,batch_size=1,collate_fn=col_fn)
M = torch.nn.Sigmoid()
pred = []
labels = []
with torch.no_grad():
    for i,batch in enumerate(dl):
        plt.figure()
        #if i == 5:
        plt.imshow(np.asarray(torch.transpose(batch['data'].squeeze().long(),0,2)))
        plt.show()
        x = batch['data']
        y = torch.unsqueeze(batch['labels'],0).float()
        #print('y ', y)
        y_hat = model.forward(x.float())
        print('y_hat ', M(y_hat)   )  
        if y_hat.numpy().tolist()[0][0] > 0.5:
            print('HAT')
            pred.append(1)
        else:
            print("NO HAT")
            pred.append(0)
        labels.append(y.numpy().tolist()[0][0])

In [19]:
precision_recall_fscore_support(labels,pred)

(array([0.55555556, 1.        ]),
 array([1. , 0.2]),
 array([0.71428571, 0.33333333]),
 array([20, 20], dtype=int64))